In [1]:
import cv2
import numpy as np
import os
import pandas as ps
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
import torch.nn as nn
from torch import optim
from torch.utils.data import ConcatDataset, DataLoader, Subset, Dataset
# This is for the progress bar.
from tqdm.auto import tqdm
from torch.nn import functional as F
from torchvision import transforms
from torchvision.utils import save_image
from model import MyDataset,UNet
from torch.utils.data import random_split
from my_metrics import calculate_metrics
import csv

from net.unetpp import NestedUNet as UNetpp

#设置设备，选择cuda
device = 'cuda' if torch.backends.mps.is_available() else 'cpu'

weight_path='params_task_3/unetpp_59.pth'
# nnunet
# model = UNet().to(device)

# unetpp
model = UNetpp(1,1).to(device)

if os.path.exists(weight_path):
    model.load_state_dict(torch.load(weight_path))
    print('successful load weight！\n')
else:
    print('model is not exist!\n')
    exit()

dicelist = []
jclist = []
hd95list = []
assdlist = []
splist = []
recalllist = []
prelist = []
for i in range(1,6):
    
    data_folder_path = f'../data/FAZ/Domain{i}/test/imgs'
    mask_folder_path = f'../data/FAZ/Domain{i}/test/mask'
    save_path = f'result/unetpp/Domain{i}'
    if not os.path.isdir('result'):
        os.mkdir('result')
    if not os.path.isdir('result/unetpp'):
        os.mkdir('result/unetpp')
    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    data_files = os.listdir(data_folder_path)
    mask_files= os.listdir(mask_folder_path)
    del_count = 0 
    for j in range(len(data_files)):
        if data_files[j-del_count].endswith(".png"):
            data_files[j-del_count] = data_files[j-del_count]
        else:
            del data_files[j-del_count]
            del_count = del_count+1
    del_count = 0      
    for j in range(len(mask_files)):
        if mask_files[j-del_count].endswith(".png"):
            mask_files[j-del_count] = mask_files[j-del_count]
        else:
            del mask_files[j-del_count]
            del_count = del_count+1

    dataset = MyDataset(data_file=data_files,mask_files=mask_files,data_folder_path=data_folder_path,mask_folder_path=mask_folder_path,test_flag=True)
    datalen = len(dataset)
    testLoader = DataLoader(dataset,batch_size=8,shuffle=True)
    all_dc,all_jc,all_hd,all_assd,all_sp,all_recall,all_pre = 0,0,0,0,0,0,0
    model.eval()
    print(f"start testing domain{i}\n")
    cur = 1
    
    for idx,(img,seg_img,fname) in enumerate(testLoader):
        img = img.to(device)
        seg_img = seg_img.to(device)
        with torch.no_grad():
             output = model(img)
        
        
        for pred,target,fname in zip(output, seg_img, fname):
            dice,jaccard,hd95_score,assd_score,sp_score,recall_score,pre_score = calculate_metrics(pred,target)
            all_dc += dice
            all_jc += jaccard
            all_hd += hd95_score
            all_assd += assd_score
            all_sp += sp_score
            all_recall += recall_score
            all_pre += pre_score

           

    dicelist.append(all_dc/datalen)
    jclist.append(all_jc/datalen)
    hd95list.append(all_hd/datalen)
    assdlist.append(all_assd/datalen)
    splist.append(all_sp/datalen)
    recalllist.append(all_recall/datalen)
    prelist.append(all_pre/datalen)
        
    print(f"the metrics of model on Domain{i} are as below")
    print(all_dc,all_jc,all_hd,all_assd,all_sp,all_recall,all_pre)
print(dicelist)

print("finish testing\n") 
transposed_lists = zip(dicelist, jclist, hd95list, assdlist, splist, recalllist, prelist)

with open('output_task_3_unetpp_59.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['domain',  'dicelist', 'jclist', 'hd95list', 'assdlist', 'splist', 'recalllist', 'prelist'])

    for i, row in enumerate(zip(dicelist, jclist, hd95list, assdlist, splist, recalllist, prelist)):
        print([i,*row])
        writer.writerow([i, *row])

In [7]:
#设置设备，选择cuda
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_param = 'unetpp'
weight_path=f'params/{model_param}_59.pth'
model = UNetpp(1,1).to(device)

if os.path.exists(weight_path):
    model.load_state_dict(torch.load(weight_path))
    print('successful load weight！\n')
else:
    print('model is not exist!\n')
    exit()
    
for i in range(1,6):
    data_folder_path = f'../data/FAZ/Domain{i}/test/imgs'
    mask_folder_path = f'../data/FAZ/Domain{i}/test/mask'
    save_path = f"result/{model_param}/Domain{i}"
    if not os.path.isdir('result'):
        os.mkdir('result')
    if not os.path.isdir(save_path):
        os.mkdir(save_path)

    data_files = os.listdir(data_folder_path)
    mask_files= os.listdir(mask_folder_path)
    del_count = 0 
    for j in range(len(data_files)):
        if data_files[j-del_count].endswith(".png"):
            data_files[j-del_count] = data_files[j-del_count]
        else:
            del data_files[j-del_count]
            del_count = del_count+1
    del_count = 0      
    for j in range(len(mask_files)):
        if mask_files[j-del_count].endswith(".png"):
            mask_files[j-del_count] = mask_files[j-del_count]
        else:
            del mask_files[j-del_count]
            del_count = del_count+1

    dataset = MyDataset(data_file=data_files,mask_files=mask_files,data_folder_path=data_folder_path,mask_folder_path=mask_folder_path,test_flag=True)
    datalen = len(dataset)
    testLoader = DataLoader(dataset,batch_size=8,shuffle=True)
    [all_dc,all_jc,all_hd,all_assd,all_sp,all_recall,all_pre] = [0,0,0,0,0,0,0]
    model.eval()
    print(f"start testing domain{i}\n")
    cur = 1
    for idx,(img,seg_img,fname) in enumerate(testLoader):
        img = img.to(device)
        seg_img = seg_img.to(device)
        with torch.no_grad():
             output = model(img)
        
        
        for pred,target,fname in zip(output, seg_img, fname):
            save_image(pred,f'{save_path}/{fname}')
    print(f"Domain{i} are done")


print("finish testing\n") 

successful load weight！

start testing domain1

Domain1 are done
start testing domain2

Domain2 are done
start testing domain3

Domain3 are done
start testing domain4

Domain4 are done
start testing domain5

Domain5 are done
finish testing

